In [1]:
root = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
save_root = "/home/shi/WorkSpace/projects/scMultiNet_Data/Step_0_data/"
import sys
sys.path.append(root )

In [2]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part
vocab_loc = root +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'QuiescenceStatus' # the label to be predicted

In [3]:

data_path = "/home/shi/WorkSpace/projects/scMultiNet_Data/Step_0_data/adata_from_cenk_raw_2024-03-24.h5ad"

In [4]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np

adata = sc.read(data_path)
print(adata)


AnnData object with n_obs × n_vars = 85749 × 33541
    obs: 'orig.ident', 'type', 'celltype', 'QuiescenceScore', 'QuiescenceStatus', 'ident'
    uns: 'X_name'
    layers: 'logcounts'


In [5]:
adata.obs['orig.ident']

Gao2021_AAACCTGCAGTGACAG                 Gao2021
Gao2021_AAACCTGGTCGAGATG                 Gao2021
Gao2021_AAACGGGGTGCACTTA                 Gao2021
Gao2021_AAACGGGTCACGGTTA                 Gao2021
Gao2021_AAAGATGAGCAGACTG                 Gao2021
                                          ...   
Qian2020_sc5rJUQ064_TTTGGTTTCTCTTATG    Qian2020
Qian2020_sc5rJUQ064_TTTGGTTTCTGCCCTA    Qian2020
Qian2020_sc5rJUQ064_TTTGTCAAGCCAGAAC    Qian2020
Qian2020_sc5rJUQ064_TTTGTCAAGGACGAAA    Qian2020
Qian2020_sc5rJUQ064_TTTGTCAGTCTTGTCC    Qian2020
Name: orig.ident, Length: 85749, dtype: category
Categories (3, object): ['Gao2021', 'Pal2021', 'Qian2020']

In [6]:
print(adata.obs["QuiescenceStatus"].unique())
print(adata.obs["celltype"].unique())


['Slow_cycling', 'Fast_cycling', 'G0_arrested']
Categories (3, object): ['Fast_cycling', 'G0_arrested', 'Slow_cycling']
['PIP.luminal', 'Cyc.Lac', 'Sec.luminal', 'Basal', 'SAA2.luminal', 'SCGB3A1.Pro', 'Cyc.Pro']
Categories (7, object): ['Basal', 'Cyc.Lac', 'Cyc.Pro', 'PIP.luminal', 'SAA2.luminal', 'SCGB3A1.Pro', 'Sec.luminal']


In [7]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="QuiescenceStatus",
                            vocab_loc="/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=None,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc="/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=0.2,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='QuiescenceStatus', vocab_loc='/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=200, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='QuiescenceStatus', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=None, random_state=2023, shuffle=True, sort_seq_batch=False)


In [8]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
screader = scReader(dataset_para=dataset_para)
# init vocab
screader.init_vocab()

# load data
screader.load_adata(loc = data_path,translate=True)



/home/shi/anaconda3/envs/scLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json
scLLM - INFO - Load data from anndata object.
scLLM - DEBUG - In original adata with gene 33541
scLLM - DEBUG - In original adata with gene 33541
scLLM - DEBUG - processing 0/16906
scLLM - DEBUG - processing 2000/16906
scLLM - DEBUG - processing 4000/16906
scLLM - DEBUG - processing 6000/16906
scLLM - DEBUG - processing 8000/16906
scLLM - DEBUG - processing 10000/16906
scLLM - DEBUG - processing 12000/16906
scLLM - DEBUG - processing 14000/16906
scLLM - DEBUG - processing 16000/16906
scLLM - INFO - create anndata in scLLM format..
scLLM - DEBU

In [9]:
screader.adata.obs["data_source"] = adata.obs['orig.ident']
screader.adata.obs["celltype"] = adata.obs['celltype']
screader.adata.obs["QuiescenceScore"] = adata.obs['QuiescenceScore']

In [10]:
screader.adata

AnnData object with n_obs × n_vars = 85749 × 16906
    obs: 'QuiescenceStatus', 'data_source', 'celltype', 'QuiescenceScore'

In [11]:
## preprocess
screader.preprocess()

scLLM - INFO - Preprocessing data with shape: (85749, 16906) ...
scLLM - INFO - Filtering cells by counts ...
scLLM - INFO - Filtered cells: 85749
scLLM - INFO - Normalizing total counts ...
scLLM - INFO - Log1p transforming ...
scLLM - INFO - Preprocessing finished.


In [12]:
# if not the first run 
# -----> 读取数据集
#from scLLM.Dataset.Reader import scReader
#data_reader = scReader(dataset_para)
# init vocab from default file loc or from list/dict given as params
#data_reader.init_vocab()
#load anndata
#data_reader.load_adata(loc = data_path,translate=False)
data_reader = screader

In [13]:
target_stimulate = "TGFb"
target_cellline = None #"A549" 'DU145', 'MCF7', 'OVCA420'
map_stimulus = False
map_cancer_type = False

print(f"选择前的数据矩阵形状 {data_reader.adata.shape}")
if map_stimulus:
    stimilus_map = data_reader.adata.obs['stimulus']==target_stimulate
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[stimilus_map,:]
    data_reader.adata = new_adata

if map_cancer_type:
    cancer_type_map = data_reader.adata.obs['CellLine']==target_cellline
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[cancer_type_map,:]
    data_reader.adata = new_adata

print(f"选择后的数据矩阵形状 {data_reader.adata.shape}")

选择前的数据矩阵形状 (85749, 16906)
选择后的数据矩阵形状 (85749, 16906)


In [14]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
# 'Slow_cycling', 'Fast_cycling', 'G0_arrested'
label_dict = {'G0_arrested':0, 'Slow_cycling':1, 'Fast_cycling':2, }

['Slow_cycling', 'Fast_cycling', 'G0_arrested']
Categories (3, object): ['Fast_cycling', 'G0_arrested', 'Slow_cycling']


In [15]:
data_reader.adata.obs[dataset_para.label_key]

Gao2021_AAACCTGCAGTGACAG                Slow_cycling
Gao2021_AAACCTGGTCGAGATG                Slow_cycling
Gao2021_AAACGGGGTGCACTTA                Slow_cycling
Gao2021_AAACGGGTCACGGTTA                Fast_cycling
Gao2021_AAAGATGAGCAGACTG                Fast_cycling
                                            ...     
Qian2020_sc5rJUQ064_TTTGGTTTCTCTTATG    Slow_cycling
Qian2020_sc5rJUQ064_TTTGGTTTCTGCCCTA    Slow_cycling
Qian2020_sc5rJUQ064_TTTGTCAAGCCAGAAC    Slow_cycling
Qian2020_sc5rJUQ064_TTTGTCAAGGACGAAA    Slow_cycling
Qian2020_sc5rJUQ064_TTTGTCAGTCTTGTCC    Slow_cycling
Name: QuiescenceStatus, Length: 85749, dtype: category
Categories (3, object): ['Fast_cycling', 'G0_arrested', 'Slow_cycling']

In [16]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'Slow_cycling': 0, 'Fast_cycling': 1, 'G0_arrested': 2}
scLLM - INFO - Discritize label QuiescenceStatus in obs_names


trainset size:  85749
no valset
{'G0_arrested': 0, 'Slow_cycling': 1, 'Fast_cycling': 2}
weights:  None


In [19]:
# add sample id to the trainset 
trainset.sample_id = list(data_reader.adata.obs.index)
trainset.q_score = list(data_reader.adata.obs["QuiescenceScore"])
trainset.celltype = list(data_reader.adata.obs["celltype"])
trainset.data_source = list(data_reader.adata.obs["data_source"])

In [20]:
import dill
# 为trainset 添加其他labels

dataset_from = "/Dataset_Cenk_Q/"
#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
#loc = save_root + dataset_from + target_task
loc = "/home/shi/WorkSpace/projects/scMultiNet_Data/Step_1_data/Dataset_Cenk_Q/"+target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,label_dict],f)